In [58]:
# Load in resources based on alpha setup
import os
from pathlib import Path
import pandas as pd
import dill
import numpy as np
import sys
from copy import deepcopy

cwd = Path(os.getcwd())
alpha_dir = cwd.parent / "env" / "alpha"
connect_dir = alpha_dir / "connectivity.csv"
poi_dir = alpha_dir / "pois.csv"
pathA_dir = alpha_dir / "pathA.csv"
pathB_dir = alpha_dir / "pathB.csv"
pathC_dir = alpha_dir / "pathC.csv"
pathD_dir = alpha_dir / "pathD.csv"
root_node_dir = alpha_dir / "root_node.csv"

if str(alpha_dir) not in sys.path:
    sys.path.append(str(alpha_dir))

from waves import wave_x, wave_y

# Useful functions
def load_connectivity_grid(csv_dir):
    df = pd.read_csv(csv_dir)
    return df.to_numpy(int)[:,1:]

def load_pois(csv_dir):
    df = pd.read_csv(csv_dir)
    return df.to_numpy(float)[:,1:]

def load_path(csv_dir):
    df = pd.read_csv(csv_dir)
    return df.to_numpy(float)[:,1:]

def import_waves(alpha_dir):
    if str(alpha_dir) not in sys.path:
        sys.path.append(str(alpha_dir))
    from waves import wave_x, wave_y

def load_root_note(root_node_dir):
    df = pd.read_csv(root_node_dir)
    return df.to_numpy(int)[:,1:]

In [59]:
connectivity_grid = load_connectivity_grid(connect_dir)
pois = load_pois(poi_dir)
pathA = load_path(pathA_dir)
pathB = load_path(pathB_dir)
pathC = load_path(pathC_dir)
pathD = load_path(pathD_dir)
root_node = load_root_note(root_node_dir)
wave_x, wave_y;

In [60]:
# Visualize
import matplotlib.pyplot as plt
import numpy as np

def plot_grid(grid, ax=None, *args, **kwargs):
    """This is a utility function that plots grids with the correct (x,y) positions"""
    if ax is None:
        plt.imshow(np.rot90(grid), *args, **kwargs, extent=(0.0, grid.shape[1], 0.0, grid.shape[0]))
    else:
        ax.imshow(np.rot90(grid), *args, **kwargs, extent=(0.0, grid.shape[1], 0.0, grid.shape[0]))

def plot_pts(pts, ax=None, *args, **kwargs):
    """This is a utility function that plots the (x,y) points specified"""
    if ax is None:
        plt.plot(pts[:,0], pts[:,1], *args, **kwargs)
    else:
        ax.plot(pts[:,0], pts[:,1], *args, **kwargs)

class Vector():
    def __init__(self, startpt, endpt):
        self.startpt = startpt
        self.endpt = endpt

def out_of_bounds(node, x_bound, y_bound):
    if node[0] >= x_bound or node[0] < 0:
        return True
    elif node[1] >= y_bound or node[1] < 0:
        return True
    return False

def sample_waves(grid, x_granularity, y_granularity):
    x_bound = grid.shape[0]
    y_bound = grid.shape[1]

    xs = np.linspace(0, x_bound, x_granularity)
    ys = np.linspace(0, y_bound, y_granularity)

    vectors = []
    for x in xs:
        for y in ys:
            if not out_of_bounds([x,y], x_bound, y_bound) and connectivity_grid[int(x),int(y)] == 1:
                startpt = np.array([x,y])
                endpt = startpt + np.array([wave_x(x), wave_y(y)])
                vectors.append(Vector(startpt, endpt))

    return vectors

def plot_vectors(vectors, ax=None, *args, **kwargs):
    """This is a utility function that plots waves as arrows following the wave gradient on a grid"""
    for vector in vectors:
        if ax is None:
            plt.plot([vector.startpt[0], vector.endpt[0]], [vector.startpt[1], vector.endpt[1]], *args, **kwargs)
        else:
            ax.plot([vector.startpt[0], vector.endpt[0]], [vector.startpt[1], vector.endpt[1]], *args, **kwargs)

In [61]:
wave_vectors = sample_waves(connectivity_grid, 75, 75)

In [62]:
fig, ax = plt.subplots(1,1,dpi=200)
plot_grid(connectivity_grid, ax, cmap='tab10_r')
plot_pts(root_node, ax, '+', color='orange')
plot_pts(pathA, ax, ls=(0, (1,2)), color='pink', lw=1)
plot_pts(pathB, ax, ls='dashed', color='purple', lw=1)
plot_pts(pathC, ax, ls='dashdot', color='tab:cyan', lw=1)
plot_pts(pathD, ax, ls=(0, (1,3)), color='tab:orange', lw=1)
plot_pts(pois, ax, marker='o', fillstyle='none', linestyle='none',color='tab:green')
plot_vectors(wave_vectors, ax, color='navy',lw=0.3)

In [63]:
# Now let's do it with waves

class AUV():
    def __init__(self, path):
        self.max_velocity = 1.
        self.path = deepcopy(path)
        self.position = self.path[0]
        # hypothesis position
        self.h_position = deepcopy(self.position)
        self.target_id = 1

    def update(self, dt):
        # Am I at my target?
        if np.allclose(self.h_position, self.path[self.target_id]):
            # Only update target if this is not the last target
            if self.target_id < len(self.path)-1:
                # New target
                self.target_id += 1
        # Delta to target
        delta = self.path[self.target_id] - self.h_position
        # Don't exceed max velocity
        if np.linalg.norm(delta) > (self.max_velocity*dt):
            diagonal = np.linalg.norm(delta)
            theta = np.arctan2(delta[1], delta[0])
            delta[0] = diagonal*np.cos(theta)
            delta[1] = diagonal*np.sin(theta)
        # Update position
        self.position += delta*dt
        self.h_position += delta*dt

dt = 0.1
auv = AUV(pathA)
actual_path = [deepcopy(auv.position)]
h_path = [deepcopy(auv.h_position)]
for i in range(1000):
    auv.update(dt)
    auv.position += np.array([ wave_x(auv.position[0])*dt, wave_y(auv.position[1])*dt ])
    actual_path.append(deepcopy(auv.position))
    h_path.append(deepcopy(auv.h_position))
actual_path = np.array(actual_path)
h_path = np.array(h_path)

In [64]:
fig, ax = plt.subplots(1,1,dpi=200)
plot_grid(connectivity_grid, ax, cmap='tab10_r')
plot_pts(root_node, ax, '+', color='orange')
plot_pts(pathA, ax, ls=(0, (1,2)), color='pink', lw=1)
plot_pts(pathB, ax, ls='dashed', color='purple', lw=1)
plot_pts(pathC, ax, ls='dashdot', color='tab:cyan', lw=1)
plot_pts(pathD, ax, ls=(0, (1,3)), color='tab:orange', lw=1)
plot_pts(pois, ax, marker='o', fillstyle='none', linestyle='none',color='tab:green')
plot_pts(actual_path, ax, ls='solid', color='black', lw=0.5)

In [65]:
# Simulate AUVs

class AUV():
    def __init__(self, path):
        self.max_velocity = 1.
        self.path = deepcopy(path)
        self.position = self.path[0]
        # hypothesis position
        self.h_position = deepcopy(self.position)
        self.target_id = 1

    def update(self, dt):
        # Am I at my target?
        if np.allclose(self.h_position, self.path[self.target_id]):
            # Only update target if this is not the last target
            if self.target_id < len(self.path)-1:
                # New target
                self.target_id += 1
        # Delta to target
        delta = self.path[self.target_id] - self.h_position
        # Don't exceed max velocity
        if np.linalg.norm(delta) > (self.max_velocity*dt):
            diagonal = np.linalg.norm(delta)
            theta = np.arctan2(delta[1], delta[0])
            delta[0] = diagonal*np.cos(theta)
            delta[1] = diagonal*np.sin(theta)
        # Update position
        self.position += delta*dt
        self.h_position += delta*dt

dt = 0.1
paths = [pathA, pathB, pathC, pathD]
auvs = [AUV(path) for path in paths]
actual_paths = [[deepcopy(auv.position)] for auv in auvs]
h_paths = [[deepcopy(auv.h_position)] for auv in auvs]

for i in range(1000):
    for id, auv in enumerate(auvs):
        auv.update(dt)
        auv.position += np.array([ wave_x(auv.position[0])*dt, wave_y(auv.position[1])*dt ])
        actual_paths[id].append(deepcopy(auv.position))
        h_paths[id].append(deepcopy(auv.h_position))

actual_paths = np.array(actual_paths)
h_paths = np.array(h_paths)

In [67]:
fig, ax = plt.subplots(1,1,dpi=200)
plot_grid(connectivity_grid, ax, cmap='tab10_r')
plot_pts(root_node, ax, '+', color='orange')
plot_pts(pathA, ax, ls=(0, (1,2)), color='pink', lw=1)
plot_pts(pathB, ax, ls='dashed', color='purple', lw=1)
plot_pts(pathC, ax, ls='dashdot', color='tab:cyan', lw=1)
plot_pts(pathD, ax, ls=(0, (1,3)), color='tab:orange', lw=1)
plot_pts(pois, ax, marker='o', fillstyle='none', linestyle='none',color='tab:green')

plot_pts(actual_paths[0], ax, ls='solid', color='pink', lw=0.5)
plot_pts(actual_paths[1], ax, ls='solid', color='purple', lw=0.5)
plot_pts(actual_paths[2], ax, ls='solid', color='tab:cyan', lw=0.5)
plot_pts(actual_paths[3], ax, ls='solid', color='tab:orange', lw=0.5)

plot_vectors(wave_vectors, ax, color='navy',lw=0.3)